# Use Machine Learning Technique to Predict pm2.5
Steps
1. Load csv data
2. Clean data and reorganize data
3. Find the best linear regression with training set, gradient, Adagrad
4. Forecast pm2.5

In [6]:
import csv
import pandas as pd
import chardet
import numpy as np

from numpy.linalg import inv
import random
import math
import sys

# Step 1: Load Data from csv
1. data = []
2. every dimension means different chemical
3. for i in range(18):
4. data.append([])

In [7]:
data = []
# each dimension save differnet medical
for i in range(18):
	data.append([])
text = open('train.csv', 'r', encoding='big5') 
row = csv.reader(text , delimiter=",")
n_row = 0
for r in row:
    # avoid header
    if n_row != 0:
        # column 3~27 denote the 24 hrs data
        for i in range(3,27):
            if r[i] != "NR":
                data[(n_row-1)%18].append(float(r[i]))
            else:
                data[(n_row-1)%18].append(float(0)) 
    n_row = n_row+1
text.close()
print("data_shape : " , np.shape(data))

data_shape :  (18, 5760)


# _Write in a function form_
* input: 
* control chemicals number: all chemicals
* control chemicals number: only pm2.5 as chemicals
* control time: all nine hours
* control time: only five hours
* regularization on/off: λ=0.1、0.01、0.001、0.0001
* output: 
* Forecast number to a file
* but print RMSE between testy and ansy
* ==============================================================
* 抽全部9小時內的污染源feature的一次項
* 抽全部9小時內pm2.5的一次項當作feature
* 抽前9小時
* 抽前5小時
* Regularization on all the weight with λ=0.1、0.01、0.001、0.0001

# Step 2: Parse Data: How to extract feature
(Pseudo code)
1. 宣告train_x 儲存前5/9小時data，以及train_y紀錄第6/10小時pm2.5值
2. for i =1月、2月......
3. 取樣每連續6/10個小時：
4. train_x.append(前5/9小時所有data)
5. train_y.append(第6/10小時pm2.5值)
6. 在train_x每筆data中加入bias

# Step 3: Linear Regression
## Original
* X = Dim( #sample , #parameters )
* w = Dim( #parameters , 1 )
* b = scaler
* Y = Xw + b
* Y = Dim( #sample , 1 )

## Rewrite bias into weight vector
* X = Dim( #sample , #parameters + 1 )
* w = Dim( #parameters+1 , 1 )
* Y = Dim( #sample , 1 )

## gradient descent
* gradient = -2 * x * (Y - Yhat)

## (Pseudo code)
1. 宣告weight vector、初始learning rate、# of iteration
2. for i_th iteration :
3. 	 y’ = train_x 和 weight vector 的 內積
4. 	 L = y’ - train_y	
5. 	 gra = 2*np.dot( (train_x)’ , L )
6. 	 weight vector -= learning rate * gra

# Step 4: Training finished! Save the training model

# Step 5: Load the testing data
* Each row: column 2~10 has data for the first 9 hours
* Every 18 row belongs to one data point (18 chemical)
* Totally the input variable is 18*9 = 162 (if add bias then 163) one chemical line up for nine hrs then another 
* Reorganize the testing data into N(=row number/18=4320/18=240) * 162

# Step 6: Use the trained/save model to predict pm2.5
1. add another dimension on x for bias (162->163)
2. (Pseudo code)
3. read test_x.csv
4. every 18 rows : 
5. test_x.append([1])
6. test_x.append(這9小時的data)
7. test_y = np.dot( weight vector, test_x)

In [127]:
def pm25(chemnum,hrnum,iteration,lrate,regular) :
    #########################################################################################################
    # Input Parameter
    # chemnum: Chemical Number, can be 18 or 1. 18 means all; 1 means only pm2.5
    # hrnum : 
    #########################################################################################################  
    
    
    
    #########################################################################################################
    # Step 2: Parse Data: How to extract feature
    #########################################################################################################    
    train_x = []
    train_y = [] 
    # max data (=how many 10 hrs overlapping windows we can see within 20 days) in each month decide h range
    # if hrnum = 9 means there are totally 10hr windons: 20*24=480 480-9=471
    # if hrnum = 5 means there are totally  6hr windows: 20*24=480 480-5=475
    windownum = 20*24 - hrnum
    for m in range(12):
        # how many overlapping windows we can see within 20 days
        for h in range(windownum):
            train_y.append(data[9][480*m+h+hrnum])
            train_x.append([])
            # number of chemical
            if chemnum == 1 :
                for t in range(hrnum):
                    train_x[windownum*m+h].append(data[9][m*480+h+t])
            if chemnum == 18:
                for i in range(chemnum):
                    for t in range(hrnum):
                        train_x[windownum*m+h].append(data[i][m*480+h+t])
    train_x = np.array(train_x)
    train_y = np.array(train_y)
    print("train_y_shape : " , np.shape(train_y))
    print("train_x_shape : " , np.shape(train_x))

    #########################################################################################################
    # Step 3: Linear Regression
    #########################################################################################################
    # Add the bias part directly to the train_x to avoid matrix addition
    train_x = np.concatenate((np.ones((train_x.shape[0],1)),train_x),axis=1)
    print("train_x_shape : " , np.shape(train_x))
    print("train_y_shape : " , np.shape(train_y))

    # Initialize weight vector(including bias)
    w = np.zeros(train_x.shape[1]) #  weight# = parameter#+1
    epsilon = math.pow(10,-8)
    totgra = np.zeros(w.shape)

    for _ in range(iteration):
        yhat = np.dot(train_x,w)
        diff = yhat - train_y
        cost = np.sum(diff**2) / len(train_x)
        cost1 = math.sqrt(cost)
        loss =  np.sum(diff**2) + regular*np.sum(w**2)
        gra =  np.dot(np.transpose(train_x),diff)
        totgra += gra**2
        ada = np.sqrt(epsilon+totgra)
        #w = w - lrate * np.divide( gra , ada )
        w = w*(1-lrate*regular/len(train_x)) - lrate * np.divide( gra , ada ) / len(train_x)
        #print ('iteration: %d | Cost: %f  ' % ( _,cost1))

    #########################################################################################################
    # Step 4: Training finished! Save the training model
    #########################################################################################################
    # save model
    np.save('pm25model.npy',w)
    # read model
    w = np.load('pm25model.npy')
    print("w_shape : " , np.shape(w))
    #print(w)
    
    #########################################################################################################
    # Step 4.5: Enlarge w into 162 dimension
    #########################################################################################################    
    bias = w[0]
    w = np.delete(w, [0])
    print("w_shape : " , np.shape(w))
    
    if chemnum==18 and hrnum==5 :
        hrmiss = 9 - hrnum
        w = np.reshape(w, (chemnum,-1))
        print("w_shape : " , np.shape(w))
        w = np.hstack((w,np.zeros((w.shape[0],hrmiss))))
        print("w_shape : " , np.shape(w))
    elif chemnum==1 and hrnum==9 :
        chemmiss = 18 - chemnum
        w = np.vstack((w,np.zeros((chemmiss,w.shape[0]))))
        print("w_shape : " , np.shape(w))
    elif chemnum==1 and hrnum==5 :
        hrmiss = 9 - hrnum
        w = np.hstack((w,np.zeros(hrmiss)))
        print("w_shape : " , np.shape(w))
        chemmiss = 18 - chemnum
        w = np.vstack((w,np.zeros((chemmiss,w.shape[0]))))
        print("w_shape : " , np.shape(w))
    
    print("w_shape : " , np.shape(w))
    w = w.flatten('F')
    print("w_shape : " , np.shape(w))
    w = np.insert(w, 0, bias)
    print("w_shape : " , np.shape(w))
    
    #########################################################################################################    
    # Step 5: Load the testing data
    #########################################################################################################
    temp = []
    # 每一個維度儲存一個“九小時的十八種化學物”資訊
    text1 = open('test.csv', 'r', encoding='big5') 
    row = csv.reader(text1 , delimiter=",")
    n_row = 0
    n_group = 0
    for r in row:
        if n_row % 18 == 0 :
            n_group += 1
            temp.append([])
        for i in range(2,11):
            if r[i] != "NR":
                temp[n_group-1].append(float(r[i]))
            else:
                temp[n_group-1].append(float(0))
        n_row = n_row+1
    text1.close()
    print("temp_shape : " , np.shape(temp))
    temp = np.array(temp)
    
    #########################################################################################################        
    # Step 6: Use the trained/save model to predict pm2.5
    #########################################################################################################    
    # Add the bias part directly to the train_x to avoid matrix addition
    testx = np.concatenate((np.ones((temp.shape[0],1)),temp),axis=1)
    print("testx_shape : " , np.shape(testx))
    
    for _ in range(testx.shape[0]):
        testy = np.dot(testx,w)
    print("testy_shape : " , np.shape(testy))
    print(type(testy))
    # add id for the first column of testy
    testy1 = np.column_stack((np.arange(testy.shape[0])+1,testy))
    testy1 = testy.tolist()
    print("testy_shape : " , np.shape(testy1))
    print(type(testy1))
    
    
    #########################################################################################################    
    # Step 7: load the answer and calculate RMSE
    #########################################################################################################    
    ans = []
    text2 = open('ans.csv', 'r', encoding='utf-8') 
    row = csv.reader(text2 , delimiter=",")
    nrow = 0
    for r in row:
        if nrow != 0:
            ans.append([])
            ans[nrow-1].append(int(r[1]))
        nrow += 1
    text2.close()
    print("ans_shape : " , np.shape(ans))
    ans = np.array(ans)
    print("ans_shape : " , np.shape(ans))
    
    # calculate the difference between ans and forecast    
    testy = testy.reshape((-1, 1))
    print("testy_shape : ", np.shape(testy))
    RMSE = math.sqrt(np.sum((testy - ans)**2))/len(testy)
    print("RMSE_shape : " , np.shape(RMSE))
    print(RMSE)
    
    return RMSE

In [143]:
pm25(1,5,10000,10,0.1)  #pm25(chemnum,hrnum,iteration,lrate,regular)

train_y_shape :  (5700,)
train_x_shape :  (5700, 5)
train_x_shape :  (5700, 6)
train_y_shape :  (5700,)
w_shape :  (6,)
w_shape :  (5,)
w_shape :  (9,)
w_shape :  (18, 9)
w_shape :  (18, 9)
w_shape :  (162,)
w_shape :  (163,)
temp_shape :  (240, 162)
testx_shape :  (240, 163)
testy_shape :  (240,)
<class 'numpy.ndarray'>
testy_shape :  (240,)
<class 'list'>
ans_shape :  (240, 1)
ans_shape :  (240, 1)
testy_shape :  (240, 1)
RMSE_shape :  ()
1.828848038800232


1.828848038800232